In [ ]:
import torch
from datasets import load_dataset

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

print(torch.__version__)
print(f"Using {device} device")

In [ ]:
dataset = load_dataset(
    "ucirvine/reuters21578", "ModApte", split="train", trust_remote_code=True
)
print(dataset[0])